In [126]:
#import dependencies 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler, OneHotEncoder



In [127]:
#set up SQL connection 
cnx = create_engine('sqlite:///alcohol_CA.db').connect() 
df = pd.read_sql_table('', cnx)

ValueError: Table  not found

In [128]:
#read in dataset
df = pd.read_csv('Data/population_pct_within_quarter_alcohol_2014.csv')  
df.head()

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Indicator_ID,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips_id,region_name,region_code,license_type,num_people_qrt,tot_people,pct_of_total
0,774,Percent of Population within 1/4 Mile of Alcoh...,2014,8,Other,CT,6001442100,4421,Alameda,6001.0,NaN,NaN,Total_licenses,0.0,0.0,NaN
1,774,Percent of Population within 1/4 Mile of Alcoh...,2014,9,Total,CT,6041122000,1220,Marin,6041.0,NaN,NaN,Total_licenses,NaN,4854.0,NaN
2,774,Percent of Population within 1/4 Mile of Alcoh...,2014,1,AIAN,CT,6065048901,489.01,Riverside,6065.0,NaN,NaN,Total_licenses,NaN,14.0,NaN
3,774,Percent of Population within 1/4 Mile of Alcoh...,2014,1,AIAN,CT,6073008336,83.36,San Diego,6073.0,NaN,NaN,Total_licenses,NaN,6.0,NaN
4,774,Percent of Population within 1/4 Mile of Alcoh...,2014,5,NHOPI,CT,6111007507,75.07,Ventura,6111.0,NaN,NaN,Total_licenses,NaN,8.0,NaN


In [129]:
df.dtypes

Indicator_ID        int64
ind_definition     object
reportyear          int64
race_eth_code       int64
race_eth_name      object
geotype            object
geotypevalue        int64
geoname            object
county_name        object
county_fips_id    float64
region_name        object
region_code       float64
license_type       object
num_people_qrt    float64
tot_people        float64
pct_of_total      float64
dtype: object

In [130]:
# Define features data
df_dropna = df.copy().dropna()
df_new = df_dropna.drop(columns=["Indicator_ID", "ind_definition", "geotype","reportyear", "geotypevalue", "geoname", "county_fips_id"])

# get categorical variables
application_cat =  df_new.dtypes[df_new.dtypes == "object"].index.tolist()
application_cat

#Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_new[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()


# Define target data
#app_df = application_df.drop(['EIN', 'NAME'], axis=1)
#X= encode_df(['pct_of_total']).values
#y = encode_df["pct_of_total"].values

,race_eth_name_AIAN,race_eth_name_AfricanAm,race_eth_name_Asian,race_eth_name_Latino,race_eth_name_Multiple,race_eth_name_NHOPI,race_eth_name_Other,race_eth_name_Total,race_eth_name_White,county_name_Alameda,...,region_name_Northeast Sierra,region_name_Northern Sacramento Valley,region_name_Sacramento Area,region_name_San Diego,region_name_San Joaquin Valley,region_name_San Luis Obispo,region_name_Santa Barbara,region_name_Shasta,region_name_Southern California,license_type_Total_licenses
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [131]:
#merge encoded with og
merged_df = df_new.merge(encode_df,left_index=True, right_index=True)
merged_df = merged_df.drop(application_cat,1)
merged_df.head()

,race_eth_code,region_code,num_people_qrt,tot_people,pct_of_total,race_eth_name_AIAN,race_eth_name_AfricanAm,race_eth_name_Asian,race_eth_name_Latino,race_eth_name_Multiple,...,region_name_Northeast Sierra,region_name_Northern Sacramento Valley,region_name_Sacramento Area,region_name_San Diego,region_name_San Joaquin Valley,region_name_San Luis Obispo,region_name_Santa Barbara,region_name_Shasta,region_name_Southern California,license_type_Total_licenses
14,1,1.0,159.0,247.0,64.37,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
15,3,1.0,2674.0,4516.0,59.21,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16,2,1.0,14243.0,22822.0,62.41,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17,4,1.0,5689.0,8092.0,70.30,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18,7,1.0,2490.0,4047.0,61.53,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [132]:
# Define target data
X= merged_df.drop(columns=['pct_of_total']).values
y = merged_df["pct_of_total"].values

In [133]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [134]:
# Create the scaler instance
X_scaler = StandardScaler()
# Fit the scaler
X_scaler.fit(X_train)
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [135]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train_scaled, y_train)
model

LinearRegression()

In [136]:
from sklearn.metrics import balanced_accuracy_score
y_predict = model.predict(X_test_scaled)

In [137]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_predict))

390.54195543746505
